<center><h1>Collaborative Filtering</h1></center>
<hr>

Name: **Firas Jolha**

Email: **f.jolha@innopolis.university**

# Install & Import Libs

In [1]:
# !pip install wget

In [52]:
import pandas as pd
from scipy import sparse
import torch
from torch import nn
from os.path import join as path_join
import numpy as np

# Data Preparation & Exploration

In [3]:
DATA_PATH = "data"
DATA_PATH = "."

train_df = pd.read_csv(path_join(DATA_PATH, "train.csv"))
test_df = pd.read_csv(path_join(DATA_PATH, "test.csv"))
train_df.head()


,userId,movieId,rating
0,1,32,3.5
1,1,47,3.5
2,1,50,3.5
3,1,253,4.0
4,1,260,4.0


In [4]:
train_df.shape, test_df.shape

((761972, 3), (190819, 3))

In [5]:
user_ids = train_df['userId'].values
movie_ids = train_df['movieId'].values
ratings = train_df['rating'].values

user_ids_index = train_df['userId'].value_counts().index
movie_ids_index = train_df['movieId'].value_counts().index

n_users = user_ids.size
n_movies = movie_ids.size

n_uq_users = user_ids_index.size
n_uq_movies = movie_ids_index.size

n_users, n_movies

(761972, 761972)

Sparse Matrices

In [38]:
# Decomposition Rank
k = 100

# Initialize Weights
weights_P = np.random.randn(n_uq_users, k)
weights_Q = np.random.randn(n_uq_movies, k)

P = sparse.coo_matrix(weights_P, shape = (n_uq_users, k), dtype = np.float)

Q = sparse.coo_matrix(weights_Q, shape = (n_uq_movies, k), dtype = np.float)
P = pd.DataFrame(weights_P, dtype = np.float, index = user_ids_index)
Q = pd.DataFrame(weights_Q, dtype = np.float, index = movie_ids_index)
# P = weights_P
# Q = weights_Q
P.shape, Q.shape

((6687, 100), (5064, 100))

In [32]:
P

array([[-0.01519227,  0.44863834, -1.09928889, ...,  0.02518814,
        -1.74119015, -1.40046257],
       [-0.52736529,  0.47732737,  1.10147088, ...,  0.7424214 ,
        -0.25420814,  0.36318748],
       [ 0.01367757, -1.17576227, -0.16411455, ...,  0.44520263,
        -0.39921133,  1.23680781],
       ...,
       [-0.98215928,  0.45349029, -0.44175254, ..., -0.63477582,
        -0.53858863,  0.02121637],
       [ 0.4729324 , -0.3141085 ,  0.70537323, ..., -0.36737836,
        -0.08216238,  1.45414548],
       [-1.25852167,  0.98028937, -0.40581161, ...,  1.62713065,
         0.46788803, -0.09443603]])

In [33]:
R = sparse.coo_matrix(
    (ratings, (user_ids, movie_ids)),
    shape=(n_users, n_movies), 
    dtype=np.float
 )

R

<761972x761972 sparse matrix of type '<class 'numpy.float64'>'
	with 761972 stored elements in COOrdinate format>

In [51]:
# Step 1
us_ids, mv_ids = R.nonzero()

# Step 2
aa = (P.loc[us_ids, :]).to_numpy()
bb = (Q.loc[mv_ids, :]).to_numpy()

# Step 3
aa.shape, bb.shape
mul = aa * bb
print(mul.shape)
# mul.shape
R_hat = P @ Q.T

# P.shape, np.max(us_ids)
# P_tau = P[us_ids,:]
# Q_tau = Q.loc[mv_ids, :]
# P_tau * Q_tau.T
# P_tau = P

(761972, 100)


In [ ]:
df = pd.MultiIndex.from_product([user_ids, movie_ids], names = ['user_id', 'movie_id'])
df

In [ ]:
df.shape

In [ ]:
# pd.DataFrame(train_df, df)

# Procedures

In [ ]:
# Users
def update_P(alpha):
  M = np.ndarray((user_ids.size, movie_ids.size))
  train_df
  P = P - alpha * ()
  pass


# Movies
def update_Q():
  pass


def als(P, Q, alpha = 0.01, iters = 100):
  for iter in range(iters):

    #Find gradient of P - Q fixed
    #Adjust P

    #Find gradient of Q - P fixed
    #Adjust Q

  pass


# Neural Collabrative Filtering Model

In [63]:
class NCA(nn.Module):
  def __init__(self):
    super().__init__()
    self.embed_user = nn.Embedding(n_uq_users, k) #, sparse=True
    self.embed_item = nn.Embedding(n_uq_movies, k)
    self.fc1 = nn.Linear(n_uq_movies + n_uq_users, 512)
    self.dropout = nn.Dropout(0.2)
    self.fc2 = nn.Linear(512, 64)
    self.fc3 = nn.Linear(64, 32)
    self.fc4 = nn.Linear(32, 8)
    self.output = nn.Linear(8,  1)
  
  def forward(self, x):
    users, items = x
    users_x = self.embed_user(users)
    items_x = self.embed_item(items)
    print(users_x.shape)
    print(movies_x.shape)
    x = torch.cat((users_x, items_x))
    print(x.shape)
    return x

  def string(self):
    return f'NCA'


# NCA()((torch.Tensor(user_ids_index), torch.Tensor(movie_ids_index)))
# Do one-hot encoding
torch.Tensor(user_ids_index).shape

torch.Size([6687])

# Visualization